In [4]:
!pip install arxiv

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
%pip install wikipedia


Note: you may need to restart the kernel to use updated packages.


In [50]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [54]:
wiki.name = "wikipedia"
wiki.description = "Search Wikipedia for general information."

In [48]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [47]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "cpu"}
)


In [46]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x70cd9b792510>, search_kwargs={})

In [45]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search","search for information about langsmith. For any questions about langsmith you must use this tool")

In [44]:
retriever_tool.name

'langsmith_search'

In [43]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
tools  = [wiki,arxiv,retriever_tool]

In [41]:
##Agents - core idea is to use a llm to choose a sequence of steps
from langchain.agents import initialize_agent,AgentType
from langchain.llms.ollama import Ollama

llm = Ollama(model = "llama3.2")

In [57]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True 
)

In [33]:
"""pip install langchainhub
"""

'pip install langchainhub\n'

In [31]:
"""from langchain import hub

prompt  = hub.pull("hwchase17/openai-functions-agent")
prompt.messages"""

'from langchain import hub\n\nprompt  = hub.pull("hwchase17/openai-functions-agent")\nprompt.messages'

In [29]:
"""##agent executer 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor""" """only needed when not initialized using initialize agent  """

'##agent executer \nfrom langchain.agents import AgentExecutor\nagent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)\nagent_executoronly needed when not initialized using initialize agent  '

In [ ]:
from langchain.agents.agent import AgentExecutor
from langchain.agents import create_react_agent
from langchain.prompts import ChatPromptTemplate

prompt_template = """
Answer the following question as best you can. You can use the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original question

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [70]:
response = agent.invoke({"input":"who is alan turing ","intermediate_steps":[]})

In [73]:
print("Agent Response:", response["output"])

TypeError: 'AgentAction' object is not subscriptable